In [1]:
from lammps import lammps
lmp=lammps()

LAMMPS (29 Aug 2024)


In [2]:
lmp.commands_string(
"""
units        metal


# GRAPHENE SHEET

dimension       3
boundary        m m p
atom_style      atomic
neighbor        0.3 bin
neigh_modify    delay 5

read_data       nx10ny55  # atom positions and simulation box size will be read from a data file to set the graphene layers


# SET THE MASS OF EACH ATOM (usually insignificant in MS simulations)

mass            1 1.0
mass            2 1.0
mass            3 1.0
mass            4 1.0
mass            5 1.0


# GRAPHENE REGION (define the boundary region, group them and set a type for each group)

region	    1 block INF 2.9 INF INF INF INF # boundary
group		lower region 1
region		2 block 38.3 INF INF INF INF INF # boundary
group		upper region 2
group		boundary union lower upper
group		mobile subtract all boundary
group       crack_lower type 4
group       crack_upper type 5

set		    group lower type 2
set		    group upper type 3

neigh_modify    exclude type 4 5  # to generate a crack


# GRAPHENE POTENTIAL

pair_style      rebo  # use REBO potential for graphene simulations
pair_coeff      * * CH.rebo C C C C C  # select the pair coefficients file and 5 C's for 5 atoms

dump            1 all custom 5000 final.dump id type xu yu zu fx fy fz
dump_modify     1 sort id


# DEFORMATION (for simulation of crack, upper and lower boundaries should be fixed)

min_style       quickmin  # choice of optimizer for energy minimization
timestep        0.003  # set the timestep

fix             fixingb boundary setforce 0.0 0.0 NULL  # fix the boundary conditions; since its 2D, z-axis is set to NULL

variable        a loop 1  # no. of iterations = 100
label           loop  # label name for the loop

displace_atoms  upper move 0.023 0.0 0.0  # how to displace the atoms in the upper region to generate a crack
displace_atoms  lower move -0.023 0.0 0.0  # how to displace the atoms in the lower region to generate a crack



minimize        1e-10 1e-10 10000 10000  # stopping criteria; the last two parameters are again the number of iterations (can be around 10,000 to 50,000); 1st parameter can also be 1e-10
run             1  # to trigger LAMMPS to run the simulation
next            a  # what to execute next
""")

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 -20) to (42.624001 135.349 20)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  2200 atoms
  read_data CPU = 0.013 seconds
220 atoms in group lower
220 atoms in group upper
440 atoms in group boundary
1760 atoms in group mobile
9 atoms in group crack_lower
9 atoms in group crack_upper
Setting atom values ...
  220 settings made for type
Setting atom values ...
  220 settings made for type
Reading rebo potential file CH.rebo with DATE: 2018-7-3
Displacing atoms ...
Displacing atoms ...

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Switching to 'neigh_m